# UPEC population structure and antimicrobial susceptibility in Norfolk, UK

### Globals

In [15]:
import pandas as pd 
import glob
import re
from os import listdir, path, rename, system
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns

from scipy.stats import hypergeom, fisher_exact, chi2_contingency
from statsmodels.stats.multitest import fdrcorrection

def rename_isolate(i):
    if i == "Reference":
        return i
    try:
        return re.search("GL\d+", i).group(0)
    except AttributeError:
        *_, GL = i.split("-")
        GL = "GL" + GL
        return rename_isolate(GL)

In [1]:
fp = r'C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\Table S1.xlsx'

### In silico typing

In [ ]:
# phylogrouping

outdir = 'dataset/phylogroup/'

def EzClerment(fp, output, script="EzClermont.py"):
    OUT = path.basename(fp).replace(".fasta", ".txt")
    system(f'python "{script}" "{fp}" 1> "{output}{OUT}" 2>&1')


def run_phylo(fp, out, script="EzClermont.py"):
    for f in glob.glob(path.join(fp, "*.fasta")):
        EzClerment(fp=f, output=out, script=script)

run_phylo("dataset/FASTA/", "dataset/phylogroup/")

### Phylogroups

In [3]:
phylogroups = pd.read_excel(r"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\Table S1.xlsx")['phylogroup']
phylogroups.value_counts(),phylogroups.value_counts(normalize=True).mul(100).round(1)

(B2    152
 D      28
 B1     19
 A       6
 F       6
 G       2
 C       2
 E       2
 Name: phylogroup, dtype: int64,
 B2    70.0
 D     12.9
 B1     8.8
 A      2.8
 F      2.8
 G      0.9
 C      0.9
 E      0.9
 Name: phylogroup, dtype: float64)

### Sequence types

In [8]:
st = pd.read_excel(r"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\Table S1.xlsx")[['ID','ST']]
st = st.fillna(0)
st['ST'] = st['ST'].astype('int32')

In [9]:
st['ST'].value_counts()

73      28
12      20
69      19
131     18
404     14
        ..
372      1
108      1
3177     1
357      1
104      1
Name: ST, Length: 75, dtype: int64

In [10]:
st['ST'].value_counts()[:8].sum() / 217 * 100

57.14285714285714

In [11]:
st['ST'].value_counts()[:8], st['ST'].value_counts(normalize=True).mul(100).round(1)[:8]

(73      28
 12      20
 69      19
 131     18
 404     14
 95      12
 127      7
 1193     6
 Name: ST, dtype: int64,
 73      12.9
 12       9.2
 69       8.8
 131      8.3
 404      6.5
 95       5.5
 127      3.2
 1193     2.8
 Name: ST, dtype: float64)

### Core genome alignment summary

In [ ]:
# core alignment summary
ca = pd.read_csv('dataset\core_alignment_summary.txt', delimiter='\t')
ca = ca[:-1]
(ca['ALIGNED'] / ca['LENGTH']).describe()
# count      217
# mean       0.843553
# std        0.055279
# min        0.728436
# 25%        0.782434
# 50%        0.858234
# 75%        0.890663
# max        0.941170

### AMR rates

In [12]:
amr = pd.read_excel(r"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\Table S1.xlsx", usecols="A,E,I,J,K,L,M,N,O")

In [13]:
amr = amr.iloc[18:,1:]

In [14]:
rx_rates = {
    "ABX" : [],
    "Overall" : [],
    "Community" : [],
    "Hospital" : []
    }

for col in amr.columns[1:]:
    rx_rates['ABX'].append(col)
    rx_rates['Overall'].append(amr.loc[:,col].value_counts(normalize=True)['R'].round(3) * 100)
    rx_rates['Hospital'].append(amr.loc[amr["Source"]=='Hospital',col].value_counts(normalize=True)['R'].round(3) * 100)
    try:
        rx_rates['Community'].append(amr.loc[amr["Source"]=='Community',col].value_counts(normalize=True)['R'].round(3) * 100)
    except KeyError:
        rx_rates['Community'].append(0)

pd.DataFrame(rx_rates)


,ABX,Overall,Community,Hospital
0,AMP,39.2,27.6,52.1
1,AUG,7.6,4.8,10.9
2,CLX,6.0,1.9,10.6
3,CIP,9.0,7.6,10.6
4,GEN,7.0,1.0,13.8
5,NIT,1.0,0.0,2.1
6,TRI,25.1,15.2,36.2


### Relationship between antibiotic resistance and source

In [19]:
def build_matrix_and_text(pheno):
    res_com = len(amr[(amr[pheno] == 'R') & (amr['Source']=='Community')])
    res_hosp = len(amr[(amr[pheno]== 'R') & (amr['Source']=='Hospital')])
    sens_com = len(amr[(amr[pheno] == 'S') & (amr['Source']=='Community')])
    sens_hosp = len(amr[(amr[pheno] == 'S') & (amr['Source']=='Hospital')])
    table = np.array([[res_com, res_hosp], [sens_com, sens_hosp]])
    chi2, p_chi, dof, _ = chi2_contingency(table, correction=False)
    return res_com, res_hosp, sens_com, sens_hosp, chi2, p_chi, dof

data = []
for r in ['AMP', 'AUG',	'GEN', 'NIT', 'TRI', 'CLX', 'CIP']:
    result = build_matrix_and_text(r)
    data.append([r]+list(result))

pd.DataFrame(data, columns=['Pheno', "res_com", 'res_hosp', 'sens_com', 'sens_hosp', 'chi2', 'p_chi', 'dof'])


,Pheno,res_com,res_hosp,sens_com,sens_hosp,chi2,p_chi,dof
0,AMP,29,49,76,45,12.500509,0.000407,1
1,AUG,5,10,100,82,2.600342,0.106841,1
2,GEN,1,13,104,81,12.575558,0.000391,1
3,NIT,0,2,105,92,2.256723,0.133035,1
4,TRI,16,34,89,60,11.551551,0.000677,1
5,CLX,2,10,103,84,6.676173,0.009771,1
6,CIP,8,10,97,84,0.549563,0.458496,1


### AMR determinants associated with phenotypic resistance

In [36]:
gene_pheno = pd.read_csv(r"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\Data\UPEC population study\gene_name_assoc.csv", index_col=0)
gene_pres_abs = pd.read_excel(r"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\code\AMR_genes_pres_abs.xlsx", index_col=0)
amr = pd.read_excel(r"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\Table S1.xlsx", usecols="A,I,J,K,L,M,N,O", index_col=0)

In [56]:
gene_pres_abs = gene_pres_abs.append(pd.Series(name='GL234', data=[0]*81, index=gene_pres_abs.columns, dtype=int))

In [30]:
pheno_genes = {pheno : [] for pheno in amr.columns[1:]}
for gene, pheno1, pheno2, pheno3 in gene_pheno.itertuples():
    pheno_genes[pheno1].append(gene)
    if not isinstance(pheno2, float):
        pheno_genes[pheno2].append(gene)
    if not isinstance(pheno3, float):
        pheno_genes[pheno3].append(gene)

In [57]:
amr.index

Index(['GL1', 'GL2', 'GL3', 'GL4', 'GL5', 'GL6', 'GL7', 'GL8', 'GL9', 'GL10',
       ...
       'GL236', 'GL237', 'GL238', 'GL239', 'GL240', 'GL243', 'GL244', 'GL245',
       'GL246', 'GL247'],
      dtype='object', name='ID', length=217)

In [62]:
gene_pres_abs[gene_pres_abs['blaCTX-M-15']==1].index

Index(['GL70', 'GL87', 'GL127', 'GL146', 'GL153', 'GL163', 'GL164', 'GL183',
       'GL204', 'GL239', 'GL240'],
      dtype='object', name='Name')

In [63]:
def build_matrix_and_text(pheno, geno):
    res_gene = len(amr[(amr[pheno] == 'R') & (amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    res_no_gene = len(amr[(amr[pheno]== 'R') & (~amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    sens_gene = len(amr[(amr[pheno] == 'S') & (amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    sens_no_gene = len(amr[(amr[pheno] == 'S') & (~amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    table = np.array([[res_gene, res_no_gene], [sens_gene, sens_no_gene]])
    oddsr, p_fisher = fisher_exact(table, alternative='two-sided')
    chi2, p_chi, dof, _ = chi2_contingency(table, correction=False)
    return res_gene, res_no_gene, sens_gene, sens_no_gene, oddsr, p_fisher, chi2, p_chi, dof

In [33]:
def get_cramer_assoc(p):
    phi2 = p / 217
    cramer_assoc = np.sqrt(phi2)
    return cramer_assoc

In [72]:
gene_pheno_assoc = {
    "gene" : [],
    "pheno" : [],
    "gene_pres_pheno_pos" : [],
    "gene_abs_pheno_pos" : [],
    "gene_pres_pheno_neg" : [],
    "gene_abs_pheno_neg" : [],
    "oddsr" : [],
    "p-value" : [],
    "chi2" : [],
    "p_chi" : [],
    "dof" : []
    }
for pheno,genes in pheno_genes.items():
    for gene in genes:
        result = build_matrix_and_text(pheno, gene)
        gene_pheno_assoc["gene"].append(gene)
        gene_pheno_assoc["pheno"].append(pheno)
        for key, val in zip(list(gene_pheno_assoc.keys())[2:], result):
            gene_pheno_assoc[key].append(val)

In [74]:
fet = pd.DataFrame(gene_pheno_assoc)

In [75]:
fet['cramer_assoc'] = fet['chi2'].apply(get_cramer_assoc)

In [76]:
p_value_corr = fdrcorrection(fet['p-value'])
fet = fet.assign(H0=p_value_corr[0], p_corrected=p_value_corr[1])

In [77]:
p_value_corr = fdrcorrection(fet['p_chi'])
fet = fet.assign(H0_chi=p_value_corr[0], p_chi_corrected=p_value_corr[1])

In [78]:
fet[(fet['H0']) & (fet['H0_chi'])].sort_values(by=['pheno', 'cramer_assoc'], ascending=False)

,gene,pheno,gene_pres_pheno_pos,gene_abs_pheno_pos,gene_pres_pheno_neg,gene_abs_pheno_neg,oddsr,p-value,chi2,p_chi,dof,cramer_assoc,H0,p_corrected,H0_chi,p_chi_corrected
59,dfrA1,TRI,22,37,3,155,30.720721,1.510279e-11,52.780564,3.729738e-13,1,0.493182,True,1.711649e-10,True,1.950940e-12
61,dfrA17,TRI,14,45,0,158,inf,3.465753e-09,40.077148,2.441282e-10,1,0.429753,True,3.366731e-08,True,1.106715e-09
63,dfrA12,TRI,7,52,0,158,inf,8.367805e-05,19.370621,1.076505e-05,1,0.298773,True,3.556317e-04,True,3.852753e-05
64,dfrA5,TRI,5,54,0,158,inf,1.307857e-03,13.705628,2.138127e-04,1,0.251316,True,4.680751e-03,True,6.321420e-04
65,dfrA14,TRI,6,53,1,157,17.773585,1.829865e-03,12.515172,4.036606e-04,1,0.240153,True,5.925278e-03,True,1.055728e-03
55,aac(3)-IIe,GEN,6,9,0,202,inf,3.700457e-08,83.097630,7.809926e-20,1,0.618820,True,2.516311e-07,True,7.586785e-19
51,aac(3)-IId,GEN,7,8,3,199,58.041667,1.710256e-07,64.841590,8.116776e-16,1,0.546634,True,8.945953e-07,True,5.017643e-15
53,aac(6')-Ib-cr5,GEN,5,10,1,201,100.500000,4.521959e-06,56.005356,7.227386e-14,1,0.508025,True,2.196380e-05,True,4.095519e-13
48,aadA5,GEN,6,9,5,197,26.266667,1.417312e-05,40.858096,1.636937e-10,1,0.433920,True,6.425147e-05,True,7.950836e-10
52,aadA2,GEN,4,11,5,197,14.327273,1.548610e-03,20.555082,5.793977e-06,1,0.307772,True,5.265273e-03,True,2.188836e-05


### Nitrofurantoin resistance determinants

In [81]:
from Bio import SeqIO

In [79]:
columns = ['qaccver', 'saccver', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'sallseqid', 'score', 'nident', 'positive', 'gaps', 'ppos', 'qframe', 'sframe', 'qseq', 'sseq', 'qlen', 'slen', 'salltitles']
blastp = pd.read_csv(r'C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\code\NIT_blastp_search.tabular', delimiter='\t', names=columns)

In [96]:
NIT_seqs = []
directory = r"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\code\NITREc\Seq\Prot"
for f in listdir(directory):
    if f.endswith('.faa'):
        with open(rf"C:\Users\carterc\OneDrive - Norwich BioScience Institutes\manuscripts\UPEC Population Study\code\NITREc\Seq\Prot\{f}") as handle:
            for record in SeqIO.parse(handle, 'fasta'):
                NIT_seqs.append({
                    "gene" : f.replace('.faa', ''),
                    "description" : record.description,
                    "seq" : str(record.seq)
                })

In [95]:
def fetch_gene_info(seq):
    for dic in NIT_seqs:
        if seq == dic['seq']:
            return dic['gene'], dic['description']

In [86]:
gpa = pd.read_csv(r'C:\Users\carterc\OneDrive - Norwich BioScience Institutes\Data\UPEC population study\dataset\gene_presence_absence.csv')

c:\Users\carterc\AppData\Local\Programs\Python\Python38-32\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (16,17,18,22,23,25,26,27,28,30,32,35,36,37,38,39,40,42,43,45,46,47,51,54,56,57,58,61,62,63,65,66,68,69,70,74,75,76,77,78,80,82,83,85,87,88,89,90,91,92,94,95,96,98,100,101,102,103,104,107,109,112,116,118,120,130,132,137,140,144,150,156,158,161,164,166,167,169,172,173,176,182,184,185,186,187,189,191,193,196,208,210,215,223,224,228,230,231) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [87]:
ID_key_dict = gpa.filter(like='GL').iloc[0].to_dict()

In [89]:
def rename_value(value):
    return re.search('[A-Z]*', value).group(0)

map_rename = lambda x : mapped_IDs[rename_value(x)]
mapped_IDs = {rename_value(v) : k for k,v in ID_key_dict.items()}
blastp = blastp.drop(blastp[blastp['saccver'].str.startswith('HNPKFMLJ')].index)
blastp["isolate"] = blastp['saccver'].map(map_rename)

In [90]:
blastp[['gene', 'description']] = pd.DataFrame(blastp['qseq'].map(fetch_gene_info).to_list(), index=blastp.index)

In [91]:
blastp_subset = blastp[(blastp['ppos'] == 100) & (blastp['pident'] == 100) & (blastp['qlen'] == blastp['slen'])]

In [94]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(blastp_subset[blastp_subset['isolate'].isin(amr[amr['NIT']=='R'].index)][['isolate', 'description', 'gene']])

      isolate                                        description     gene
1342    GL116  NfsA nfsA|IN09|2|287989-288480|-|ICL2020_00865...  NfsA__R
5150     GL14  NfsA nfsA|EC958|NZ_HG941718.1|900522-901244|+|...  NfsA__S
10266     GL4  NfsB nfsB|IN06|1|705493-705774|+|ICL2020_00712...  NfsB__R
10619   GL160  NfsB nfsB|IN07|2|122188-122619|+|ICL2020_00466...  NfsB__R
15019    GL14  NfsB nfsB|ATCC25922|NZ_CP009072.1|4640270-4640...  NfsB__S
18477    GL14  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18479    GL13  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18484    GL12  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18488     GL3  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18489    GL11  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18494     GL2  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18498     GL5  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18501     GL4  RibE ribE|ATCC25922|NZ_